In [1]:
# To enable plotting graphs in Jupyter notebook
%matplotlib inline
# Enable auto complete
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np

# Import ploting modules
import matplotlib.pyplot as plt
import seaborn as sns

# Import model modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.compose import ColumnTransformer

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
des_df = pd.read_csv('LCDataDictionary.csv')

In [5]:
des_df = des_df[des_df.Description.notna()]
des_df = des_df[des_df.LoanStatNew.notna()]

In [6]:
data = pd.read_csv('lending_csv_v3.zip', compression='zip')
data.head()

C:\ProgramData\Anaconda3\envs\quovantis\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (49,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1604328,70811807,NaN,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,E3,HR Generalis,10+ years,MORTGAGE,61000.0,Verified,Feb-2016,Charged Off,n,NaN,debt_consolidation,Debt consolidation,235xx,VA,27.15,1.0,Oct-2004,665.0,669.0,0.0,5.0,NaN,12.0,0.0,23676.0,81.1,41.0,f,0.0,0.0,4696.20,4696.20,763.00,1307.48,0.0,2625.72,472.6296,Aug-2016,347.59,NaN,Dec-2016,544.0,540.0,0.0,47.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,298836.0,0.0,3.0,2.0,9.0,8.0,28171.0,88.0,1.0,3.0,870.0,85.0,29200.0,1.0,5.0,2.0,12.0,24903.0,611.0,70.9,0.0,0.0,121.0,130.0,11.0,8.0,6.0,11.0,44.0,11.0,5.0,3.0,2.0,5.0,2.0,12.0,18.0,6.0,17.0,5.0,12.0,0.0,0.0,0.0,3.0,82.9,50.0,0.0,0.0,337444.0,51847.0,2100.0,31884.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,241274,982256,NaN,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,B5,Kmart,10+ years,MORTGAGE,52645.0,Not Verified,Oct-2011,Charged Off,n,Borrower added on 10/07/11 > misc. expenses ...,other,personnal,606xx,IL,12.47,0.0,Apr-2001,670.0,674.0,0.0,50.0,NaN,5.0,0.0,5910.0,46.9,18.0,f,0.0,0.0,822.35,822.35,529.19,203.63,0.0,89.53,0.8000,Sep-2012,67.09,NaN,Oct-2016,659.0,655.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,N

In [56]:
missing_col = set(data.columns) - set(des_df.LoanStatNew)
missing_col_df = data.loc[:, missing_col]
invalid_value_col = list(missing_col_df.columns[missing_col_df.isna().sum() > 240000])
df = data.drop(invalid_value_col + [
    'member_id', 'id', 'Unnamed: 0', 'pymnt_plan', 'zip_code', 'next_pymnt_d', 'acc_now_delinq','policy_code','application_type',
    'disbursement_method', 'dti_joint', 'annual_inc_joint', 'desc', 'next_pymnt_d', 'mths_since_rcnt_il', 'all_util', 'max_bal_bc',
    'open_rv_24m', 'total_cu_tl', 'total_bal_il', 'open_il_24m', 'open_il_12m', 'disbursement_method', 'hardship_flag', 
    'collection_recovery_fee', 'il_util', 'open_act_il', 'title', 'emp_title','chargeoff_within_12_mths'
], axis=1)
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,tot_coll_amt,tot_cur_bal,open_acc_6m,open_rv_12m,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,debt_settlement_flag
0,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,E3,10+ years,MORTGAGE,61000.0,Verified,Feb-2016,Charged Off,debt_consolidation,VA,27.15,1.0,Oct-2004,665.0,669.0,0.0,5.0,NaN,12.0,0.0,23676.0,81.1,41.0,f,0.0,0.0,4696.200000,4696.20,763.00,1307.48,0.0,2625.72,Aug-2016,347.59,Dec-2016,544.0,540.0,0.0,47.0,0.0,298836.0,0.0,1.0,29200.0,1.0,2.0,12.0,24903.0,611.0,70.9,0.0,121.0,130.0,11.0,8.0,6.0,11.0,44.0,11.0,5.0,3.0,2.0,5.0,2.0,12.0,18.0,6.0,17.0,5.0,12.0,0.0,0.0,0.0,3.0,82.9,50.0,0.0,0.0,337444.0,51847.0,2100.0,31884.0,N
1,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,B5,10+ years,MORTGAGE,52645.0,Not Verified,Oct-2011,Charged Off,other,IL,12.47,0.0,Apr-2001,670.0,674.0,0.0,50.0,NaN,5.0,0.0,5910.0,46.9,18.0,f,0.0,0.0,822.350000,822.35,529.19,203.63,0.0,89.53,Sep-2012,67.09,Oct-2016,659.0,655.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N
2,29075.0,29075.0,29075.0,36 months,27.34,1192.31,F,F5,4 years,MORTGAGE,120000.0,Source Verified,May-2016,Charged Off,home_improvement,OH,10.40,0.0,Dec-2001,670.0,674.0,2.0,30.0,NaN,12.0,0.0,5648.0,60.1,20.0,f,0.0,0.0,3532.770000,3532.77,1626.15,1906.62,0.0,0.00,Aug-2016,1192.31,Dec-2017,594.0,590.0,1.0,37.0,194.0,141941.0,1.0,3.0,9400.0,10.0,7.0,6.0,12904.0,980.0,84.4,0.0,75.0,172.0,4.0,4.0,4.0,4.0,NaN,0.0,NaN,2.0,2.0,4.0,2.0,3.0,3.0,9.0,12.0,4.0,12.0,0.0,0.0,0.0,4.0,89.5,50.0,0.0,0.0,164709.0,48895.0,6300.0,55084.0,N
3,16000.0,16000.0,16000.0,36 months,8.39,504.27,A,A5,10+ years,MORTGAGE,89000.0,Not Verified,Oct-2014,Charged Off,debt_consolidation,MD,18.85,0.0,Aug-1989,685.0,689.0,0.0,30.0,NaN,10.0,0.0,13662.0,88.7,19.0,f,0.0,0.0,15649.100000,15649.10,11616.41,1991.12,0.0,2041.57,Jan-2017,504.27,Dec-2017,559.0,555.0,0.0,NaN,0.0,32271.0,NaN,NaN,15400.0,NaN,NaN,2.0,3586.0,660.0,93.2,0.0,146.0,302.0,5.0,5.0,0.0,55.0,NaN,NaN,30.0,0.0,3.0,6.0,3.0,4.0,6.0,8.0,13.0,6.0,10.0,0.0,0.0,0.0,1.0,89.5,66.7,0.0,0.0,54454.0,32271.0,9700.0,39054.0,Y
4,35000.0,35000.0,35000.0,36 months,18.49,1273.96,D,D2,10+ years,MORTGAGE,600000.0,Verified,Apr-2013,Charged Off,major_purchase,SC,11.76,3.0,Jun-1997,680.0,684.0,1.0,1.0,NaN,12.0,0.0,1630.0,12.4,31.0,f,0.0,0.0,20148.830000,20148.83,10102.81,6458.64,0.0,3587.38,Jul-2014,1296.34,Oct-2016,559.0,555.0,0.0,46.0,0.0,1250817.0,NaN,NaN,13100.0,NaN,NaN,4.0,104235.0,11470.0,12.4,0.0,193.0,174.0,25.0,13.0,3.0,25.0,1.0,2.0,1.0,1.0,4.0,4.0,4.0,8.0,17.0,4.0,11.0,4.0,12.0,0.0,2.0,0.0,0.0,66.0,25.0,0.0,0.0,1412518.0,336990.0,13100.0,438745.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [57]:
for col in df.isna().sum().sort_values(ascending=False).index:
    if df[col].isna().sum()> 0:
        print(col)
#         print('1:- ',des_df[des_df.LoanStatNew == col])
        print('2:- ',data[col].isna().sum())
#         print('3:- \n', data[col].value_counts())
#         print(data[col].describe())
        print('\n\n')

mths_since_last_record
2:-  248430



mths_since_recent_bc_dlq
2:-  227630



mths_since_last_major_derog
2:-  220106



mths_since_recent_revol_delinq
2:-  198404



inq_fi
2:-  181856



open_rv_12m
2:-  181856



open_acc_6m
2:-  181856



inq_last_12m
2:-  181856



mths_since_last_delinq
2:-  149774



mths_since_recent_inq
2:-  38287



num_tl_120dpd_2m
2:-  26726



mo_sin_old_il_acct
2:-  23632



emp_length
2:-  18079



pct_tl_nvr_dlq
2:-  15158



avg_cur_bal
2:-  15123



num_tl_30dpd
2:-  15119



num_accts_ever_120_pd
2:-  15119



total_il_high_credit_limit
2:-  15119



total_rev_hi_lim
2:-  15119



tot_cur_bal
2:-  15119



tot_coll_amt
2:-  15119



mo_sin_old_rev_tl_op
2:-  15119



mo_sin_rcnt_rev_tl_op
2:-  15119



mo_sin_rcnt_tl
2:-  15119



num_tl_90g_dpd_24m
2:-  15119



tot_hi_cred_lim
2:-  15119



num_actv_bc_tl
2:-  15119



num_bc_tl
2:-  15119



num_tl_op_past_12m
2:-  15119



num_rev_tl_bal_gt_0
2:-  15119



num_il_tl
2:-  15119



num_rev_accts
2:

## Unusually null columns

removing rows with majority Null Data. 

In [58]:
such_columns  = ['num_accts_ever_120_pd','tot_hi_cred_lim','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl',
                 'total_rev_hi_lim','num_tl_op_past_12m','num_tl_90g_dpd_24m','num_tl_30dpd','num_rev_tl_bal_gt_0','num_rev_accts','num_op_rev_tl'
                ,'tot_cur_bal','num_bc_tl','total_il_high_credit_limit'
,'num_actv_rev_tl'
,'num_actv_bc_tl'
,'tot_coll_amt'
,'num_il_tl']

In [59]:
df = df[df[such_columns].notnull().all(1)]

In [89]:
df.mths_since_last_major_derog = df.mths_since_last_major_derog.fillna(0)
df.mths_since_last_record = df.mths_since_last_record.fillna(0)
df.mths_since_recent_bc_dlq = df.mths_since_recent_bc_dlq.fillna(0)
df.mths_since_recent_revol_delinq = df.mths_since_recent_revol_delinq.fillna(0)
df.mths_since_last_delinq = df.mths_since_last_delinq.fillna(0)
df.open_acc_6m = df.open_acc_6m.fillna(0.0).astype(int)
df.inq_fi = df.inq_fi.fillna(0.0).astype(int)
df.inq_last_12m = df.inq_last_12m.fillna(0.0).astype(int)
df.open_rv_12m = df.open_rv_12m.fillna(0.0).astype(int)
df.open_rv_12m = df.open_rv_12m.fillna(0.0).astype(int)
df.mths_since_recent_inq = df.mths_since_recent_inq.fillna(0).astype(int)
df.num_tl_120dpd_2m = df.num_tl_120dpd_2m.fillna(0).astype(int)
df.mo_sin_old_il_acct = df.mo_sin_old_il_acct.fillna(df.mo_sin_old_il_acct.mean())
df.revol_util = df.revol_util.fillna(df.revol_util.mean())



df.emp_length = df.emp_length.fillna(df.emp_length.mode()).astype(str)
df.pct_tl_nvr_dlq = df.pct_tl_nvr_dlq.fillna(100).astype(int)
df.avg_cur_bal = df.avg_cur_bal.fillna(df.avg_cur_bal.median()).astype(int)
df.num_op_rev_tl = df.num_op_rev_tl.fillna(df.num_op_rev_tl.mean()).astype(int)
df.dti = df.dti.fillna(df.dti.mean()).astype(int)
df.last_credit_pull_d = df.last_credit_pull_d.fillna('Dec-2017')
df.last_pymnt_d = df.last_pymnt_d.fillna('Dec-2017')


C:\ProgramData\Anaconda3\envs\quovantis\lib\site-packages\pandas\core\generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\ProgramData\Anaconda3\envs\quovantis\lib\site-packages\pandas\core\generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [61]:
df.term = df.term.str.replace(' months', '').astype(int)

In [90]:
for col in df.isna().sum().sort_values(ascending=False).index:
    if df[col].isna().sum()> 0:
        print(col)
        print('1:- ',des_df[des_df.LoanStatNew == col].values)
        print('2:- ',data[col].isna().sum())
#       print('3:- \n', data[col].value_counts())
#       print(data[col].describe())
        print('\n\n')

bc_util
1:-  [['bc_util'
  'Ratio of total current balance to high credit/credit limit for all bankcard accounts.']]
2:-  13437



percent_bc_gt_75
1:-  [['percent_bc_gt_75'
  'Percentage of all bankcard accounts > 75% of limit.']]
2:-  13360



bc_open_to_buy
1:-  [['bc_open_to_buy' 'Total open to buy on revolving bankcards.']]
2:-  13278



mths_since_recent_bc
1:-  [['mths_since_recent_bc'
  'Months since most recent bankcard account opened.']]
2:-  13098





In [92]:
print(df['bc_open_to_buy'].value_counts())
print(df['bc_open_to_buy'].describe())

0.0        5344
500.0       258
2000.0      237
1000.0      233
3000.0      202
           ... 
76562.0       1
20334.0       1
19333.0       1
54840.0       1
37021.0       1
Name: bc_open_to_buy, Length: 43348, dtype: int64
count    282022.000000
mean       9348.094826
std       14300.312144
min           0.000000
25%        1306.000000
50%        4269.000000
75%       11273.000000
max      337415.000000
Name: bc_open_to_buy, dtype: float64


# Converted loan term data type from string to number

## Filling column value with default 0 as per assumption it should be 0 month

## Verify column with nan value more than 60%

In [ ]:
col_with_missing_values= [col for col in df.loc[:,df.columns[df.isna().sum() > 200000]].columns if col in set(des_df.LoanStatNew)]

In [ ]:
df = df.drop(col_with_missing_values, axis=1)

In [ ]:
len(df.columns[df.isna().sum() > 0])

In [ ]:
df.columns[df.isna().sum()> 150000]
des_df.LoanStatNew.isin(df.loc[:,df.columns[df.isna().sum() > 150000]].columns)
des_df.loc[: , des_df.columns.isin(df.loc[:,df.columns[df.isna().sum() > 150000]].columns)]

In [ ]:
for col in df.select_dtypes(exclude='object').columns:
    df[col].fillna(df[col].mean(), inplace=True)
for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode(), inplace=True)


In [ ]:
converted_df = df.sample(frac=0.05, random_state=1)
converted_df_x = converted_df.drop('loan_status', axis=1)

In [ ]:
def rfe(X, y):
        '''
        Returns a dataframe with only a subset of features/columns retained
        '''
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.feature_selection import RFE
        est = RandomForestClassifier(n_estimators=100, oob_score=True, max_features=5)
#        selectK = SelectKBest(score_func = f_classif, k=45)
        select_rfe = RFE(estimator=est, n_features_to_select=50, step=0.15)
        select_k=select_rfe
        
        select_k.fit(X,y)
        return select_k
#         K_featnames = feature_names[selectK_mask]
#         print ("reduced RFE features:")
#         print(K_featnames)
#         reduced_df = pd.read_csv(filename, index_col=0)
#         reduced_df = reduced_df[reduced_df.columns[selectK_mask]]

#         return reduced_df
selected_k = rfe(converted_df_x,converted_df.loan_status)


In [ ]:
converted_df

## Trying with Automatically Select the Number of Features 

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

In [ ]:
X = converted_df_x
y = converted_df.loan_status
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


In [ ]:
X.select_dtypes(include='object').columns

# Label Encoding

In [ ]:
cleanup_nums = {
    "grade": {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7},
    "sub_grade": {"A1": 1, "A2": 2, "A3": 3, "A4": 4, "A5": 5, "B1": 6, "B2": 7, "B3": 8, "B4": 9, "B5": 10, "C1": 11, "C2": 12, "C3": 13, "C4": 14, "C5": 15, "D1": 16, "D2": 17, "D3": 18, "D4": 19, "D5": 20, "E1": 21, "E2": 22, "E3": 23, "E4": 24, "E5": 25, "F1": 26, "F2": 27, "F3": 28, "F4": 29, "F5": 30, "G1": 31, "G2": 32, "G3": 33, "G4": 34, "G5": 35}
}

In [ ]:
import json
json.dumps({k:i+1 for i,k in enumerate(list(X.sub_grade.sort_values().unique()))})

In [ ]:
# X.earliest_cr_line.unique()
# des_df[des_df.LoanStatNew=='earliest_cr_line'].Description.values

X.verification_status.unique()
# des_df[des_df.LoanStatNew=='initial_list_status'].Description.values
# X = X.drop('next_pymnt_d', axis=1)

In [ ]:
# create pipeline
model = SVR(kernel='linear')
rfe = RFECV(model, step=5, cv=cv, scoring='accuracy', n_jobs=5, min_features_to_select = 50)

In [ ]:
rfe.fit(X, y)
print('Optimal number of features: {}'.format(rfecv.n_features_))
# model = DecisionTreeClassifier()
# pipeline = Pipeline(steps=[('s',rfe),('m',model)])

# n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=4, error_score='raise')
# report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

## Deleteing columns who have more than 50% null value

In [ ]:
df = data.loc[:, list(data.columns[data.isna().sum() < 150000])]
df.head()

In [ ]:
df = df.drop(['Unnamed: 0', 'id'], axis=1)
df.head()

In [ ]:
set(df.select_dtypes(exclude='object').columns)

In [ ]:
cln = df.select_dtypes(exclude='object').columns
for cl in cln:
    sns.boxplot(df.chd, df[cl])
    plt.title(f'Column({cl}) vs chd')
    plt.show()